# Matching upwind and downwind of the power plant

This Jupyter notebook aims to estimate the percentage of time each person was downwind of roadways and match residences predominatly upwind and downwind of the power plant.

Reference is [here](https://academic.oup.com/aje/article/193/12/1720/7693601?login=false). 

We need ArcGIS Pro and Python.

# Set-up

In [ ]:
## Install ArcGIS Pro
## Create a virtual environment due to the compatibility of `arcpy`
#python3 --version
#conda create -n arcgispro_env python=3.11 arcpy=3.4 -c esri
#conda activate arcgispro_env
#conda install arcpy=3.4 -c esri
#python -m ipykernel install --user --name arcgispro_env --display-name "Python (arcgispro_env)"
## Change the kernel to 'arcgispro_env'

# ArcGIS API requires cryptography <42
# pip uninstall cryptography
# pip install cryptography==41.0.7

In [ ]:
import os
import math
import numpy as np
import pandas as pd
import datetime
import time
from multiprocessing import Pool 
import arcpy
import osmnx as ox
from pathlib import Path
import sys

In [3]:
arcpy.env.overwriteOutput = True

In [4]:
## Create my project in ArcGIS in the project folder
## Then, set-up my ArcGIS workspace
arcpy.env.workspace = r"C:\Users\shiroa1\OneDrive - VUMC\plant_closure_project\plant_closure_project\Default.gdb"

# Download Wind Vectors

Download hourly u and v wind vectors from the European Centre for Medium-Range Weather Forecasts (ECMWF; fifth major global reanalysis produced by ECMWF [ERA5] reanalysis version 5). This download process cannot be done under VUMC network!! Thus, I downloaded all the files locally.

Please note we do not consider wind speed.

In [5]:
#conda config --add channels conda-forge
#conda install cdsapi
#https://confluence.ecmwf.int/display/CKB/How+to+install+and+use+CDS+API+on+Windows
#url: https://cds.climate.copernicus.eu/api
#key: 9becc2c0-1b45-444d-9739-3872e6f999fc
##Open notepad
##Paste the two lines above
##Save as '.cdsapirc' in 'C:\Users\User\' 
#conda activate arcgispro_env
#pip3 install cdsapi 
import cdsapi
client = cdsapi.Client()

In [6]:
# Summary: 
# 1) download daily 10m u- and v- wind vector components from the ERA5 reanalysis product
# 2) calculate daily wind speed from u and v wind vectors
# 3) reference: https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation

In [7]:
# download hourly wind components for one 24 hour period
def downloadHourlyDataOneData(year,month,day,filename):
    d = client.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': ['reanalysis'],
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind',
        ],
        'year': str(year),
        'month': str(month).zfill(2),
        'day': str(day).zfill(2),
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'format': 'grib',
        # [North, West, South, East] in degrees
        # Boundary of the Nashville-Davidson country
        'area': [
            36.40550, -87.05470, 35.9677, 
            -86.51559,
        ],
    },
    filename)


In [8]:
# download one year of wind components (hourly temporal resolution)
def downloadSingleYearData(year):
    yearFolder = WIND_VECTOR_FOLDER + str(year) + "/"
    if not(os.path.exists(yearFolder)): 
        os.mkdir(yearFolder)

    # for each month in the year
    for month in list(range(1,13)):
        curDays = DAYS_IN_MONTH[month-1]
        if(year % 4 == 0 and month == 2): # adjust days in Feb for leap year if necessary
            curDays +=1
        
        # for each day in the month
        for day in list(range(1,curDays+1)):
            outputFile = yearFolder + f"{year}_{str(month).zfill(2)}_{str(day).zfill(2)}.grib" # .grib file is a multivariate raster
            if not os.path.exists(outputFile):
                downloadHourlyDataOneData(year,month,day,outputFile)

In [9]:
# create string to extract time value from multivariate timeseries raster
def createTimeRangeString(year,month,day,hour):
    prefix = "StdTime "
    dateString = str(year) + "-" + str(month).zfill(2) + "-" + str(day).zfill(2)
    hourString = 'T' + str(hour-1).zfill(2) + ":00:00"
    return(prefix + dateString + hourString)

In [10]:
# extract u- and v- wind components for a single hour from a 
# a multivariate timeseries raster and create a wind direction
def convertComponentsToDirectionOneHour(inputFile,year,month,day,hour,outputFile):
    tempUComponent = PARENT_FOLDER + "temp_u_" + f"temp_u_{year}.crf" # .crf file is a raster in ArcGIS 
    tempVComponent = PARENT_FOLDER + "temp_v_" + f"temp_u_{year}.crf" # .crf file is a raster in ArcGIS

    # extract u- and v-componenets from multivariate raster
    queryString = createTimeRangeString(year,month,day,hour)
    arcpy.md.SubsetMultidimensionalRaster(
        inputFile,tempUComponent,COMPONENT_DICT['u_comp'],
        "BY_VALUE", None, queryString,
        '', '', '', None, ''
    )
    arcpy.md.SubsetMultidimensionalRaster(
        inputFile,tempVComponent,COMPONENT_DICT['v_comp'],
        "BY_VALUE", None, queryString,
        '', '', '', None, ''
    )

    # create wind direction raster and save
    windDirection = calcWindDirection(tempUComponent,tempVComponent)
    windDirection.save(outputFile)
    arcpy.management.DefineProjection(outputFile, PROJECTION)

    # cleanup temporary files
    arcpy.Delete_management(tempUComponent)
    arcpy.Delete_management(tempVComponent)
# Check .grib file using ArcGIS
# Open ArcGIS and my project
# Search 'Add Data' 

In [11]:
# create wind direction raster from u- and v- wind component rasters
def calcWindDirection(uFile,vFile):
    u_comp = arcpy.Raster(uFile)
    v_comp = arcpy.Raster(vFile)
    uArr = arcpy.RasterToNumPyArray(u_comp)
    vArr = arcpy.RasterToNumPyArray(v_comp)
    windArr = 180 + (180/math.pi)*np.arctan2(uArr,vArr) # wind direction array # wind direction=180+(180/π)*atan2(u,v)
    windRaster = newRaster = arcpy.NumPyArrayToRaster(
        # ensures the new wind direction raster is aligned with the original u- and v-component rasters in terms of location and resolution
        windArr,
        arcpy.Point(u_comp.extent.XMin,u_comp.extent.YMin), # specifies the lower-left corner of the raster
        u_comp.meanCellWidth # provides the cell size
    )
    return(windRaster)

In [12]:
# convert hourly u- and v- wind components for an entire year to wind direction estimates
# save wind direction estimates as .tif files, one file for each hour in the year
def convertComponentsToDirectionOneYear(year):
    
    # setup
    inputFolder = WIND_VECTOR_FOLDER + f"{year}/"
    outputFolder = WIND_DIRECTION_FOLDER + f"{year}/"
    if not(os.path.exists(outputFolder)):
        os.mkdir(outputFolder)
    
    # for each month in the year
    for month in list(range(1,13)): 
        curDays = DAYS_IN_MONTH[month-1]
        if(year % 4 == 0 and month == 2): # adjust days in Feb for leap year if necessary
            curDays +=1
        
        # for each day in the month
        for day in list(range(1,curDays+1)):
            print(f"Processing day {day}")

            # for each hour of the day
            for hour in list(range(1,25)):
                outputFile = outputFolder + f"{year}_{str(month).zfill(2)}_{str(day).zfill(2)}_{str(hour).zfill(2)}.tif"

                if not os.path.exists(outputFile):
                    inputFile = inputFolder + f"{year}_{str(month).zfill(2)}_{str(day).zfill(2)}.grib"
                    convertComponentsToDirectionOneHour(inputFile,year,month,day,hour,outputFile)

# Implementation

In [13]:
# Parameters
# Create each folder in advance
# Input file paths should be absolute
PARENT_FOLDER = 'C:/Users/shiroa1/OneDrive - VUMC/plant_closure_project/WIND_FOLDER/'
WIND_VECTOR_FOLDER = PARENT_FOLDER + "wind_surfaces/wind_components/"
WIND_DIRECTION_FOLDER = PARENT_FOLDER + "wind_surfaces/wind_direction/"
DAYS_IN_MONTH = [31,28,31,30,31,30,31,31,30,31,30,31]
LEAP_YEARS = [2008,2012,2016]
COMPONENT_DICT = {
    'u_comp': '10U@SFC',
    'v_comp': '10V@SFC'
}
YEARS = list(range(2005,2006)) # 2005,2006; 2006,2007; 2010,2011; 2011,2012; 2012,2013; 2013,2014, 2015,2016; 2016,2017; 2017,2018
PROJECTION = "GEOGCS['GCS_Coordinate_System_imported_from_GRIB_file',DATUM['D_unknown',SPHEROID['Sphere',6367470.0,0.0]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]"
N_PROCESSES = 4

In [ ]:
# Implementation
if __name__ == '__main__':
    for year in YEARS:
        downloadSingleYearData(year)
    with Pool(processes=N_PROCESSES) as pool:
        res = pool.map(convertComponentsToDirectionOneYear,YEARS)

In [14]:
# If the code above takes a lot of time
downloadSingleYearData(2011) #should be run on the local

2025-10-22 18:40:02,219 INFO Request ID is 63825a0b-0833-4ca2-a31f-5eb32fcd1095
2025-10-22 18:40:02,401 INFO status has been updated to accepted
2025-10-22 18:40:11,714 INFO status has been updated to running
2025-10-22 18:40:24,700 INFO status has been updated to successful


63523f98b975dd902bd81aedfa8230bf.grib:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

2025-10-22 18:40:29,408 INFO Request ID is 3a8a0694-6372-4c52-a553-65220518c384
2025-10-22 18:40:29,568 INFO status has been updated to accepted
2025-10-22 18:41:02,888 INFO status has been updated to successful


40be7077c1946e1a765b28999af94b55.grib:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

2025-10-22 18:41:05,334 INFO Request ID is 32242224-d6e3-4060-9465-58e9bf9091c3
2025-10-22 18:41:05,486 INFO status has been updated to accepted
2025-10-22 18:41:14,252 INFO status has been updated to running
2025-10-22 18:41:27,249 INFO status has been updated to successful


...

f22e99318888c3f2e544d06fcc98f06e.grib:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

2025-10-22 21:48:15,187 INFO Request ID is 9f32ba47-b2d8-43c8-9157-5ebced21709f
2025-10-22 21:48:15,508 INFO status has been updated to accepted
2025-10-22 21:48:29,794 INFO status has been updated to running
2025-10-22 21:48:37,734 INFO status has been updated to successful


2842587d2806efa5d6579242d819e1ea.grib:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

2025-10-22 21:48:40,501 INFO Request ID is 8483a05a-1c87-4163-9109-a4a040a87ed1
2025-10-22 21:48:40,648 INFO status has been updated to accepted
2025-10-22 21:48:49,363 INFO status has been updated to running
2025-10-22 21:49:02,329 INFO status has been updated to successful


c4880a42fa0af7816e79d67601c312f0.grib:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

In [ ]:
# If the code above takes a lot of time
convertComponentsToDirectionOneYear(2010) #should be run on the server

# Partion by year

Please prepare shape files in the 'Birth_Addresses_Wind/births_by_year', each containing residential addresses registered for each year.

For example, 'Birth_Addresses_Wind/births_by_year/birth_2010.shp'.

Child ID: 'idx'

In [5]:
# Summary: given hourly wind rasters and annual residences, extract hourly
#          values for two years and combine them into a single csv.

# Steps include:
# 1) extracting values to points
# 2) extracting date and hour from filename
# 3) combining results to create a single comprehensive csv

In [6]:
# check if 24 rasters are available for one day.  If so, return
# array with filepaths to the rasters

def createDailyFileArray(windDailyPrefix):
    dailyArray = []
    for hour in range(1,25):
        dailyArray.append([windDailyPrefix + "_" + str(hour).zfill(2) + ".tif", str(hour).zfill(2)])
    nCorrect = 0
    for arr in dailyArray:
        if(os.path.exists(arr[0])):
            nCorrect +=1
    if nCorrect == 24:
        return(dailyArray)
    return([]) 

In [7]:
# convert arcgis attribute table to pandas dataframe
def arcgis_table_to_df(in_fc):
    print("converting to dataframe")
    OIDFieldName = arcpy.Describe(in_fc).OIDFieldName # Get object ID field name
    final_fields = [field.name for field in arcpy.ListFields(in_fc)] # List all field names
    data = [row for row in arcpy.da.SearchCursor(in_fc,final_fields)] # Fetch all rows
    fc_dataframe = pd.DataFrame(data,columns=final_fields) # Set OID field as index
    fc_dataframe = fc_dataframe.set_index(OIDFieldName,drop=True)
    valsList = []

    # for each hour in the day, prefix the variable name with a letter to avoid 
    # a bug issue later with variables that start with numbers
    for i in range(1,25):
        fc_dataframe[str(i).zfill(2)] = fc_dataframe["F" + str(i).zfill(2)].astype(int)
        valsList.append(str(i).zfill(2))
    valsList.append("idx") # you can rename the child ID
    fc_dataframe = fc_dataframe[valsList]
    return fc_dataframe

In [8]:
# for all births within a single year, extract 2 years of hourly wind directions
# I changed the window from '-1 to 1' to '0 to 2'
def extractPointVals(year):
    birthShapefile = BIRTH_FOLDER + "births_" + str(year) + ".shp"
    masterBirthFile = "memory/births_" + str(year)

    # create a temp copy of the birth shapefile to prevent accidentally altering the original
    arcpy.CopyFeatures_management(birthShapefile, masterBirthFile)
    outputFolder = WIND_FOLDER + str(year)
    if not(os.path.exists(outputFolder)):
        os.mkdir(outputFolder)

    # for 2 years of wind direction, starting with the year of birth.
    for curYear in range(year,year+2): # I changed here to use the code in our child cohort
        windFolder = WIND_DIRECTION_FOLDER + str(curYear) + "/"

        # for each month in the year and each day in the month, extract raster values to points and 
        # save the results in csv format
        for month in range(1,13):
            for day in range(1,32):
                outputFile = outputFolder + "/" + str(curYear) + "_" + str(month) + "_" + str(day) + ".csv"
                if not(os.path.exists(outputFile)):
                    birthCopy = "memory/t_births_" + str(year) + str(month) + str(day) 
                    arcpy.CopyFeatures_management(masterBirthFile, birthCopy)
                    windDayPrefix = windFolder + str(curYear) + "_" + str(month).zfill(2) + "_" + str(day).zfill(2) 
                    dailyArray = createDailyFileArray(windDayPrefix)
                    if(len(dailyArray)==24):
                        try:
                            arcpy.sa.ExtractMultiValuesToPoints(birthCopy,dailyArray)
                            pandasDf = arcgis_table_to_df(birthCopy)
                            pandasDf.to_csv(outputFile,index=False)
                            arcpy.Delete_management(birthCopy)
                        except Exception as e:
                            print("couldn't extract value to points for file %s: %s" %(windDayPrefix,str(e)))

In [9]:
# given a filename with the date in it, extract the month and day
def getMonthDayFromFilename(filename):
    monthStart = filename.find("_") # find the first
    monthEnd = filename[monthStart+1:].find("_")
    month = filename[monthStart+1:][:monthEnd]
    dayStart = filename.rfind("_") # find the last
    dayEnd = filename[dayStart+1:].find(".csv")
    day = filename[dayStart+1:][:dayEnd]
    return([month,day])      

In [10]:
# given a csv of measurements, denormalize the variables and store in dataframe format
def transformCSVtoDF(folder,csv,curYear):
    df = pd.read_csv(folder + "/" + csv)
    curMonth,curDay = getMonthDayFromFilename(csv)
    hour,day,month,year,wnd_dir,master_id = [[] for x in range(6)]

    # denormalize dataset (denomralize means 'converting wide to long format')
    for curHour in range(1,25):
        tempVals = list(df[str(curHour).zfill(2)])
        nVals = len(tempVals)
        wnd_dir += tempVals
        hour += [curHour-1 for x in range(nVals)]
        month += [curMonth for x in range(nVals)]
        day += [curDay for x in range(nVals)]
        year += [curYear for x in range(nVals)]
        master_id += list(df['idx']) # you can rename the child ID
    df = pd.DataFrame({
        'year':year,
        'month':month,
        'day':day,
        'hour':hour,
        'wnd_dir':wnd_dir,
        'master_id':master_id    
    })
    return(df)

In [11]:
# given a single csv for each day, combine all records for a year of births into a single 
# comprehensive csv
def combineAnnualCSVs(year):
    outputFile = WIND_FOLDER + "/comb_" + str(year) + ".csv"
    if(os.path.exists(outputFile)):
        return
    inputFolder = WIND_FOLDER + str(year)
    csvsToCombine = os.listdir(inputFolder)
    pandasData = transformCSVtoDF(inputFolder,csvsToCombine[0],year)
    index = 1
    allPandas = []

    # for each daily csv file of wind direction estimates, add to the dataframe
    for file in csvsToCombine[1:]:
        if(index%30==0):
            allPandas.append(pandasData)
            pandasData = transformCSVtoDF(inputFolder,file,year) # I guess this was a typo (original: 'csvsToCombine[0]', not 'file')
            # https://github.com/larkinandy/Matching_HEI_4970/blob/main/wind%20metrics/scripts/partitionWindByYear.py#L77
            # See line 173
        else:
            pandasData = pd.concat([pandasData,transformCSVtoDF(inputFolder,file,year)])
        print(index)
        index+=1
    for tempData in allPandas:
        pandasData = pd.concat([pandasData,tempData])
    pandasData.to_csv(outputFile,index=False)

# Implementation

We can change the number of processes (Here, I used 8).

In [13]:
# Parameters
YEARS = list(range(2010,2011))
#YEARS = list(range(2007,2017))
WIND_FOLDER = 'C:/Users/shiroa1/OneDrive - VUMC/Geocoding/WIND_FOLDER/'
PARENT_FOLDER = WIND_FOLDER
BIRTH_FOLDER = 'C:/Users/shiroa1/OneDrive - VUMC/Geocoding/Birth_Addresses_Wind/births_by_year/'
WIND_DIRECTION_FOLDER = PARENT_FOLDER + "wind_surfaces/wind_direction/"

In [ ]:
# Implementation
if __name__ == '__main__':
    with Pool(processes=8) as pool: # len(YEARS) is the default
        res = pool.map(extractPointVals,YEARS)
    
    for year in list(range(2010,2011)): # not 'YEARS', we should add +1 to the end year
        combineAnnualCSVs(year)

In [17]:
# If the code above takes a lot of time
extractPointVals(2010) # should delete 'WIND_FOLDER/2010' folder beforehand

In [ ]:
# If the code above takes a lot of time
combineAnnualCSVs(2010)

# Create wind matrix

Originally, GPU-based coding was written, however, our server is not able to use GPU, but rely on CPU. 

First, I used my laptop with a GPU to understand what is going on under the hood. Reference is [Here](https://github.com/larkinandy/Matching_HEI_4970/blob/main/wind%20metrics/scripts/createWindMarix.py)

Then, I wrote the Python script for a CPU-based server setting.

In [5]:
# Summary: using hourly wind data, create daily estimates of hours upwind for each of 360 radial degrees
#          around each maternal residence in a yearly subset of the Texas cohort
# Steps to create wind estimates include :
# 1) loading hourly wind estimates from storage
# 2) reformatting data for matrix algebra operations on GPUs
# 3) push data to GPU memory
# 4) calculate number of days downwind for each radial degree using GPUS
# 5) push wind matrix back to RAM 
# 6) save wind matrix to storage

In [6]:
# I decreased batch size for GPUs with less RAM
# My GPU: GPU 0: NVIDIA RTX A2000 8GB Laptop GPU, GPU 1: Interl(R) Iris(R) Xe Graphics

### GPU-based

I have written CPU-based coding for our server; however, GPC on my local works faster.

In [8]:
# Installs only the CUDA runtime libraries inside your Conda environment
#conda install cudatoolkit
# Install nvvm.dll and the full CUDA toolkit from NVIDIA
# https://developer.nvidia.com/cuda-downloads
# locate CUDA-related executables
# Command Prompt: where nvcc

In [10]:
import os
import math
# conda remove numpy numba --force
# numpy should be grade-downed
# conda install numpy=1.26 numba=0.59
import numpy as np
import pandas as pd
import time
from multiprocessing import Pool 
from pathlib import Path
import sys

In [11]:
from __future__ import division # In Python 3, this future import is unnecessary because true division (/) is the default
from numba import cuda, float32

In [12]:
# Read hourly wind estimates extracted for all residences in the birth year
# Output is array of tuples.
# Each tuple contains sorted hourly wind estimates and unique ids
def readRawData(year):
    windTuples = [] # each tuple contains wind estimates and unique ids for 1000 births
    windData= pd.read_csv(WIND_FOLDER + "/comb_" + str(year) + ".csv")
    print("finished loading wind data")
    uniqueIds = list(set(windData['master_id'])) # all unique ids in the curr batch
    numIds = len(uniqueIds) 
    curIndex = 0

    # for each 1000 unique ids, get wind records for those ids, sort the ids and wind records
    # in the same order, and create a tuple.  Adjust the month variable to be relative to the first 
    # month of coverage (january year before birth)
    while(curIndex < numIds):
        subsetids = uniqueIds[curIndex: min(curIndex+1000,numIds)] # get next 1000 unique ids
        subsetData = windData[windData['master_id'].isin(subsetids)] # get records for this subset of ids
        subsetData.sort_values(by=['master_id'],inplace=True) # sort wind records by unique ids
        subsetids.sort() # sort unique ids
        
        # for records in the year of the birth, add 12 to the months value.  This makes the month variable
        # relative to the first month of coverage (january year before birth)
        # IN OUT PROJECT, WE SHOULD NOT DO THAT!!!!!
        #tempVals = (subsetData['year'] - year +1)*12 # 
        tempVals = (subsetData['year'] - year)*12 # 
        subsetData['month'] = subsetData['month'] + tempVals
        
        windTuples.append((subsetData,subsetids))
        curIndex+=1000 # go to next batch of 1000 unique ids
        print(curIndex)
    print("completed partitioning wind data")
    return(windTuples)

In [13]:
# create a uniqueId->index map.  Called in combination with the python map function
def mapIds(idToMap):
    return masterList.index(idToMap)

In [14]:
# cuda function.  Must be compiled before runtime (@cuda.jit decorator)

# rearrange data so hourly records in the matrix are organized by:
#      matrix[hourIndex][dayIndex][monthIndex][uniqueIdIndex]
# INPUTS:
#    idList (numpy array) - indeces for unique ids
#    hour (numpy arrray) - hour of the wind direction measures
#    day (numpy array) - day of the wind direction measures
#    month (numpy array) - month of the wind direction measures
#    wind_dir (numpy array) - wind direction measures
#    outData (numpy array) - stores reorganized wind direction measures
#    maxThread (int) - number of wind direction measures
@cuda.jit
def rearrangeData(idList,hour,day,month,wnd_dir,outData,maxThread):
    tid = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    if(tid < maxThread):
        outData[hour[tid],day[tid]-1,month[tid]-1,idList[tid]] = wnd_dir[tid]
    

# calculate number of hours each radial degree is upwind of the maternal residence
# results are stored in place in the output dataset
# INPUTS:
#    inputData (numpy array) - hourly wnd direction matrix
#    outputData (numpy array) - where results are stored in place
@cuda.jit
def calcDailyDownwind(inputData,outputData):
    
    
    # inline function.  Must be within compiled function so it can be sent to the 
    # GPU driver and implemented by each streaming multiprocessor at runtime
    # test if a radial degree is within the +/- 15 degree angular window for being downwind
    # INPUTS:
    #    val1 (int) - radial degree
    #    val2 (int) - wind direction
    # OUTPUTS:
    #     True if radial degree is within 15 degrees of wind direction. False otherwise.
    def isDownwind(val1,val2):
        diff1 = val1 - val2 if val1-val2 > 0 else (val1-val2 + 360)
        diff2 = val2 - val1 if val2-val1 >0 else (val2 - val1 + 360)
        minDiff = min(diff1,diff2)
        if(minDiff < 15):
            return 1
        return 0
    
    # each of the 360 degrees is assigned a unique thread id along the x axis of the cuda block
    angle = cuda.threadIdx.x
    
    # cuda has thousands of threads per sm along the x axis.  No need to go beyond 360
    if(angle >= 360):
        return

    # important that these dimensions/indeces match those assigned in the funtion 'rearrange data'
    day = cuda.blockIdx.y
    month = cuda.blockIdx.x
    station = cuda.blockIdx.z

    # for each hour in 24 hours of a day, test if the current radial degree is downwind of the wind direction
    # add to the hours downwind if yes, do not change hoursDownwind otherwise
    hoursDownwind = 0
    for hourIndex in range(24):
        wndAngle = inputData[hourIndex,day,month,station]
        if(wndAngle>=0):
            hoursDownwind += isDownwind(angle,wndAngle)
        else:
            hoursDownwind = -999
    
    # WARNING: IT IS ESSENTIAL THAT EACH THREAD HAS GUARANTEED EXCLUSIVE ACCESS TO IT's INDEX IN THE 
    # outputData matrix.  One thread, one index exactly.

    # update the output dataset in place
    outputData[station,month,day,angle] = hoursDownwind

In [15]:
# Installs only the CUDA runtime libraries inside your Conda environment
#conda install cudatoolkit
# Install nvvm.dll and the full CUDA toolkit from NVIDIA
# https://developer.nvidia.com/cuda-downloads
# locate CUDA-related executables
# Command Prompt: where nvcc
# I did not have to define the path, but reinstalled Numpy and Cuda

In [16]:
# constants necessary for GPU oeprations.  MUST BE SET/HARD-CODED BEFORE RUNTIME!!!!
N_ANGLES = 360
N_BIRTHS = 624 # input data is preprocessed into batch sizes of 1000
thread_x_dim = math.ceil(360/128)*128 # number of threads per streaming multiprocessor
thread_dim = (thread_x_dim,1) # cuda requires 2d tuple
block_x_dim = 24 # months in 2 years, one of the dimensions of the output wind matrix
block_y_dim = 31 # max days in month, one of the dimensions of the output wind matrix
block_z_dim = N_BIRTHS # number of children
block_dim = (block_x_dim,block_y_dim,block_z_dim)
TOTAL_THREADS = thread_x_dim*block_x_dim*block_y_dim*block_z_dim

In [17]:
#BIRTH_FOLDER = 'C:/Users/shiroa1/OneDrive - VUMC/Geocoding/Birth_Addresses_Wind/births_by_year/'
WIND_FOLDER = 'C:/Users/shiroa1/OneDrive - VUMC/Geocoding/WIND_FOLDER/'
YEAR = 2010

In [18]:
windTuples = readRawData(YEAR)
index = 0

finished loading wind data
1000
completed partitioning wind data


In [19]:
windPartitions = 'C:/Users/shiroa1/OneDrive - VUMC/Geocoding/WIND_FOLDER/windPartitions/' + str(YEAR)
if not os.path.exists(windPartitions):
    os.makedirs(windPartitions)

In [20]:
masterIds = 'C:/Users/shiroa1/OneDrive - VUMC/Geocoding/WIND_FOLDER/masterIds/' + str(YEAR)
if not os.path.exists(masterIds):
    os.makedirs(masterIds)

In [ ]:
# for each batch of 1000 maternal residences
# rearrange data into matrix organized by day, month, year, and unique id
# calculate daily number of hours downwind for all 360 degrees 
# save data as npy file
for tupleVal in windTuples:
    outputFile = WIND_FOLDER + 'windPartitions/' + str(YEAR) + '/w_' + str(index) + '.npy' # should remove an extra space, not '/w_ '
    
    # no need to process data if output file was already created
    if not(os.path.exists(outputFile)):
        print("starting index %i" %(index))

        windData, masterList = tupleVal[0], tupleVal[1]

        # get index for each unique id
        inputIds = list(map(mapIds,windData['master_id']))

        # transfer data from RAM to GPU memory.  Part of this process
        # includes creating variables on the GPU
        inputList = cuda.to_device(np.array(inputIds))
        cuda.synchronize()
        inputHour = cuda.to_device(np.array(windData['hour']))
        cuda.synchronize()
        inputDay = cuda.to_device(np.array(windData['day']))
        cuda.synchronize()
        inputMonth = cuda.to_device(np.array(windData['month']))
        cuda.synchronize()
        inputDir = cuda.to_device(np.array(windData['wnd_dir']))
        cuda.synchronize()        
        testInput = np.full((24,31,24,N_BIRTHS),-999,np.int16)
        outData = cuda.to_device(testInput)
        cuda.synchronize()
        nThreads = len(windData['hour'])
        cuda.synchronize()
        nBlocks = math.ceil(nThreads/thread_x_dim)
        cuda.synchronize()

        # rearrange wind direction so its organized in a matrix by day,month,year,unique id
        rearrangeData[(nBlocks,1),(thread_x_dim,1)](inputList,inputHour,inputDay,inputMonth,inputDir,outData,nThreads)
        cuda.synchronize()
        print("finished rearranging data")

        blankFile = np.full((N_BIRTHS,24,31,360),-999,np.float64) # blank output matrix.  CUDA writes in place
        cuda.synchronize()

        station_mem = cuda.to_device(blankFile) # version of blank file to store on the GPU
        cuda.synchronize()

        # calcualte number or hours downwind for each of 360 degrees, for each day, each maternal residence
        calcDailyDownwind[block_dim,(thread_dim)](outData,station_mem)
        print("calcualted daily averages")
        cuda.synchronize()
     
        # transfer results from GPU to RAM
        resStation = station_mem.copy_to_host()

        # convert results to pandas dataframe and save 
        with open(outputFile, 'wb') as f:
            np.save(f, resStation)
        masterDF = pd.DataFrame({
            'masterIds':masterList
        })
        masterDF.to_csv(WIND_FOLDER + "masterIds/" + str(YEAR) + "/id_" + str(index) + ".csv",index=False)
    index+=1    

In [29]:
demo = np.load("C:/Users/shiroa1/OneDrive - VUMC/Geocoding/WIND_FOLDER/windPartitions/2010/w_0.npy")
demo.shape

(624, 24, 31, 360)

# Convert hourly upwind and downwind data in CSV files to a rose-wind shapefile

This section sholud be done on the server due to the privacy policy.

In [22]:
# Summary: create rose-wind shapefiles (1 degree radial resolution) for each maternal address
# Steps to create rose wind shapefiles include :
# 1) load 2 years of daily wind directions from storage
# 2) load maternal residence metadata including coordinates, birth date, and conception date
# 3) restrict daily wind directions to [conception date, birth date]
# 4) sum daily wind directions for pregnancy period 
# 5) create road wind shapefiles in ArcGIS

In [30]:
sucessfulImport = False

In [31]:
# needed when using the ArcGIS license for a large # of parallel threads
while(sucessfulImport == False):
    try:
        import arcpy
        from arcpy.ia import * 
        from arcpy import env
        from arcpy.sa import *
        arcpy.env.overwriteOutput = True
        sucessfulImport = True
    except Exception as e:
        print("couldn't import arcgis: %s" %(str(e)))
        time.sleep(1)

In [32]:
parentFolder = 'C:/Users/shiroa1/OneDrive - VUMC/Geocoding/WIND_FOLDER/' # main folder containing all files of interest
testInputFolder = parentFolder   # folder where all csv files and shapefiles are located
inputShapeFolder = BIRTH_FOLDER = 'C:/Users/shiroa1/OneDrive - VUMC/Geocoding/Birth_Addresses_Wind/births_by_year/' # folder where shapefiles are located
windFilename = "shpfl.shp"                              # name for the temporary wind vector shapefile
year = 2010
inputCSVFilepath = testInputFolder + "/comb_" + str(year) + ".csv"           # CSV containing wind direction and date stamps
CSV_DICT = ["latitude","longitude"]     # values are stored in latitude and longitude
WIND_FIELD = "UpWind"                      # attribute field in temp and temp2 files
LATITUDE_FIELD = "latitude"                 # name of the attribute field in the shapefile containing latitude coords
LONGITUDE_FIELD = "longitude"               # name of the attirbute field in the shapefile containing longitude coords
NUM_TIME_SAMPLES = 365*4

In [33]:
# create a 360 element array, with one element for each angular degree
def createAngularMatrix():
    angMatrix = [0] *360
    return(angMatrix)

In [34]:
# determine how often each of the 360 angular degrees are upwind from an air monitor.  For this calculation,
# upwind is defined as being within the windDegree number of degrees from directly upwind

In [35]:
def addWindExposure(windDirection,angMatrix):
    windDegree = 23     # tolerance for how many degrees deviation is allowed from directly upwind to still be classified as upwind
    
    if(windDirection == 360):        # if the wind value is 360, change to 0, which is the angular equivalent of 360
        windDirection = 0
    
    if(windDirection <windDegree):   # if the wind direction is less than the windDegree value, some values less than 0 (i.e. around 350 or so) are also upwind
        runWindComp(0,windDirection+windDegree,angMatrix) # calculate the number of days the degree angles to the right of due north are upwind
        leftSideMin = 360+windDirection-windDegree  # determine the minimum angle to the left of due north that is upwind
        runWindComp(leftSideMin,360,angMatrix) # calculate number of days the degree angles to the left of due north are upwind

    elif(windDirection > 338):       # if the wind direction is slightly to the left of due north, some values to the right of due north are also upwind
        runWindComp(windDirection-windDegree,360,angMatrix) # calculate the number of days the degree values to the left of due north are upwind
        rightSideMax = 360-windDirection + windDegree # determine the maximum angle to the right of due north that is upwind
        runWindComp(0, rightSideMax,angMatrix) # calculate the number of days the degree angles to the right of due north are upwind

    else:
        runWindComp(windDirection-windDegree,windDirection+windDegree,angMatrix) #calculate the number of days degree angles are upwind
            

In [36]:
# for each angle element in angMatrix, increment if they are upwind for the given wind direction

In [37]:
def runWindComp(windMinDegree,windMaxDegree,angMatrix):
    
    for angle in range(0,360):
        if(angle >= windMinDegree and angle <= windMaxDegree):
            angMatrix[angle]+=1

In [38]:
# find the index in the two input datasets where the latitude and longitude values match

In [39]:
def matchLatLong(infoArr,latLongArr):
    i=0
    for latVal in infoArr[0]:
        if(latVal == latLongArr[0] and infoArr[1][i] == latLongArr[1]):
            return(i)
        i+=1

# Create windrose shapefiles

This section creates 'WIND_FOLDER/outputShapefiles/year/year.shp' files.
This section should be done on the server due to the confidentiality.

In [ ]:
# Steps to create rose wind shapefiles include :
# 1) load 2 years of daily wind directions from storage
# 2) load maternal residence metadata including coordinates, birth date, and conception date
# 3) restrict daily wind directions to [conception date, birth date]
# 4) sum daily wind directions for pregnancy period 
# 5) create road wind shapefiles in ArcGIS

In [70]:
# needed when using the ArcGIS license for a large # of parallel threads
sucessfulImport = False
while(sucessfulImport == False):
    try:
        import arcpy
        from arcpy.ia import * 
        from arcpy import env
        from arcpy.sa import *
        arcpy.env.overwriteOutput = True
        sucessfulImport = True
    except Exception as e:
        print("couldn't import arcgis: %s" %(str(e)))
        time.sleep(1)

In [71]:
sr = arcpy.SpatialReference()   # WGS84 spatial reference
sr.factoryCode = 4326
sr.create()

In [72]:
# the following components are used for the Haversine formula.  For more details go to
# http://www.movable-type.co.uk/scripts/latlong.html
# https://www.eol.ucar.edu/content/wind-direction-quick-reference
EARTH_RADIUS = 6378140.0                # used to calculate wind direction
distance = 55000.0/EARTH_RADIUS         # used to calculate bearing and wind direction
N_ANGLES = 360

In [73]:
# calculate latitude and longitude coordinates on the outside of the buffer region
def calcCoords(origLat,origLong,bearing,distance):    
    latRadians = origLat*0.0174533          # convert latitude from degrees to radians
    longRadians = origLong*0.0174533        # convert longitude from degrees to radians
    
    # calculate the new latitude coordinate for a point that is 'distance' units away using the haversine formula (see weblink above)
    newLatit = math.asin(math.sin(latRadians)*math.cos(distance) + math.cos(latRadians)*math.sin(distance)*math.cos(bearing))/0.0174533
    
    # calculate the new longitude coordinate for a point that is 'distance' units away using the haversine formula
    longitP1 = math.sin(bearing)*math.sin(distance)*math.cos(latRadians)
    longitP2 = math.cos(distance) - math.sin(latRadians)*math.sin(newLatit*0.0174533)    
    newLongit = longRadians + math.atan2(longitP1,longitP2)
    newLongit = newLongit/0.0174533
    return([newLatit,newLongit])

In [74]:
# create a triangle polygon in arcgis using 3 coordinate points.  The triangle will represent one angular degree of coverage
def calcTriangle(wndDrt,strtLat, strtLong,distance,inputFilename,colNames,colVals):
    bearing1 = ((wndDrt + 0.5)%360.0)*0.0174533       # center of triangle + 0.5 degrees
    bearing2 = ((wndDrt - 0.5)%360.0)*0.0174533       # center of triangle - 0.5 degrees
    airMonitorLoc = arcpy.Point(strtLong,strtLat)       # defnie the air monitor location as one point of the triangle
    coordP1 = calcCoords(strtLat,strtLong,bearing1,distance)  # create another coordinate point
    upPoint1 = arcpy.Point(coordP1[1],coordP1[0])
    coordP2 = calcCoords(strtLat,strtLong,bearing2,distance)  # create another coordinate point
    upPoint2 = arcpy.Point(coordP2[1],coordP2[0])
    # Create a polygon geometry
    array = arcpy.Array([airMonitorLoc,upPoint1,upPoint2])
    polygon = arcpy.Polygon(array)
    # Open an InsertCursor and insert the new geometry
    cursor = arcpy.da.InsertCursor(inputFilename, ['SHAPE@'] + colNames)
    rowVals = tuple([polygon] + list([colVals]))
    cursor.insertRow(rowVals)
    # Delete cursor object
    del cursor

In [75]:
# add syntax necessary to create multiple attributes in an ArcGIS attribute table
def mapCreateFieldNames(fieldName):
    return( [fieldName,"Short"])

In [76]:
# given a specific date and year, get indeces the daily wind direction is stored in the 2 year wind direciton array
def getBirthIndexes(inDate,bYear):
    datetime_object = datetime.datetime.strptime(inDate, '%Y-%m-%d')
    # WE SHOULD NOT DO THAT IN OUR EV PROJECT
    # as our focus in on infancy rather than pregnancy
    #monthIndex = (datetime_object.month + 11) if datetime_object.year == bYear else (datetime_object.month -1)
    monthIndex = (datetime_object.month + 11) if datetime_object.year == bYear+1 else (datetime_object.month -1)
    dayIndex = datetime_object.day - 1
    return([monthIndex,dayIndex])

In [77]:
# given data for a single birth, restrict wind values to the infancy period and create a wind rose matrix
def processAnnualVals(windData,bdate,cdate,byear):
    bmonth,bday = getBirthIndexes(bdate,byear)
    cmonth,cday = getBirthIndexes(cdate,byear)
    startVals = np.sum(np.maximum(windData[cmonth][cday:],0),axis=0)
    startVals += np.sum(np.maximum(windData[bmonth][:bday],0),axis=0)
    for monthData in windData[cmonth+1:bmonth]:
        startVals += np.sum(np.maximum(monthData,0),axis=0)
    return(startVals)

In [78]:
# given maternal residence information and a wind rose matrix, create a rose wind shapefile in ArcGIS
def calcAnnualShapefile(lat,lon,year,angleVals,uniqueId):
    colNames = ['sum_' + str(year)]
    fieldNames = list(map(mapCreateFieldNames,colNames))
    yearShapefile = uniqueId + ".shp"
    arcpy.CreateFeatureclass_management(OUTPUT_SHAPEOLDER,yearShapefile,"POLYGON",'#','#','#',sr)
    arcpy.management.AddFields(OUTPUT_SHAPEOLDER + "/" + yearShapefile,fieldNames)
    for angle in range(360):
        calcTriangle(angle+0.5,lat,lon,distance,OUTPUT_SHAPEOLDER + "/" + yearShapefile,colNames,angleVals[angle])

In [79]:
# given 2 years of daily wind direction and all metadata about the maternal residence and birth date, 
# perform all steps necessary to create a wind rose shapefile
def processSingleResidence(dataTuple):
    outputFile = OUTPUT_SHAPEOLDER + "/" + dataTuple[5] + ".shp"
    if not(os.path.exists(outputFile)):
        sumWindVals = processAnnualVals(dataTuple[0],dataTuple[1],dataTuple[2],YEAR)
        calcAnnualShapefile(dataTuple[3],dataTuple[4],YEAR,sumWindVals,dataTuple[5])

In [83]:
# given multiple sources of raw data, repackage the data to facilitate parallel processing 
def prepParallel(windData,idData,personalData):
    parallelArray = []
    for i in range(idData.count()[0]):
        curId = list(idData.iloc[i])
        curPersonal = personalData[personalData['idx'].isin(curId)] # child_id
        if(int(curPersonal.count()[0])>0):
            curTuple = (windData[i],list(curPersonal['bdate'])[0],list(curPersonal['cdate'])[0],
            list(curPersonal['lat'])[0],list(curPersonal['lon'])[0],curId[0]) # originally, 'b_lat' and 'b_long'
            parallelArray.append(curTuple)
    return(parallelArray)

In [84]:
YEAR = 2010
WIND_FOLDER = 'C:/Users/shiroa1/OneDrive - VUMC/Geocoding/WIND_FOLDER/'
PARENT_FOLDER = WIND_FOLDER
masterIdFolder = PARENT_FOLDER + "/masterIds/" + str(YEAR) + '/'
windPartitions = PARENT_FOLDER + "/windPartitions/" + str(YEAR) + "/"
BIRTH_FOLDER = 'C:/Users/shiroa1/OneDrive - VUMC/Geocoding/Birth_Addresses_Wind/births_by_year/'
BIRTH_DATA = pd.read_csv(BIRTH_FOLDER + str(YEAR) + ".csv")
OUTPUT_SHAPEOLDER = PARENT_FOLDER + "outputShapefiles/" # This should be 'outputShapefiles/', not '/outputShapefiles/'
if not (os.path.exists(OUTPUT_SHAPEOLDER)):
    os.mkdir(OUTPUT_SHAPEOLDER)    
OUTPUT_SHAPEOLDER = PARENT_FOLDER + "outputShapefiles/" + str(YEAR) # This should be 'outputShapefiles/', not '/outputShapefiles/'
if not (os.path.exists(OUTPUT_SHAPEOLDER)):
    os.mkdir(OUTPUT_SHAPEOLDER)  

In [ ]:
if __name__ == '__main__':
    masterIds = os.listdir(masterIdFolder)
    for index in range(len(masterIds)):
        windData = np.load(windPartitions + "w_" + str(index) + ".npy") # There was an unnecessary space here
        idData = pd.read_csv(masterIdFolder + "id_" + str(index) + ".csv")
        parallelData = prepParallel(windData,idData,BIRTH_DATA)
        print("finished prepping data")
        with Pool(processes=4) as pool: # originally 64
            pool.map(processSingleResidence,parallelData)

finished prepping data


# Create road categories

Based on the traffic density shapefile, derive annual road network subsets based on traffic cutoffs.

In [ ]:
# remove unwanted attribute fields from the shapefile attribute table
def trimFields(shapeFilepath):
    fieldNames = [f.name for f in arcpy.ListFields(shapeFilepath)]
    dropFields = []
    for name in fieldNames:
        if name != "route_id" and name != "geometry": # Originally, name != "FID" and name != "Shape"
            #dropFields.append(name)
            try:
                arcpy.DeleteField_management(shapeFilepath,[name])
            except exception as e:
                print(str(e))

In [ ]:
# for each year and traffic category, create subset files
def createAnnualFiles(year):
    masterList = []
    
    # master road network file is different for 2007-2009 and 2010-2016
    masterRoadFile = ROAD_FOLDER + "rhino_vmt_2010_2016.shp" if year >= 2010 else ROAD_FOLDER + "rhino_vmt_1995_2009.shp"
    ROAD_ANNUAL_FOLDER = ROAD_FOLDER + str(year)
    if not os.path.exists(ROAD_ANNUAL_FOLDER):
        os.mkdir(ROAD_ANNUAL_FOLDER)
    
    # for each general traffic level cutoff, create a subset shapefile of roads that satisfy the range
    for cutoffIndex in range(len(aadtLevels)-1):
        outputShapefile = ROAD_ANNUAL_FOLDER + "/aadt" + str(cutoffIndex) + ".shp"
        masterList.append(outputShapefile)
        if not os.path.exists(outputShapefile):
            lowerCutoff = aadtLevels[cutoffIndex]
            upperCutoff = aadtLevels[cutoffIndex+1]
            var = "aadt" + str(year) 
            expression = var + " > " + str(lowerCutoff) 
            arcpy.Select_analysis(masterRoadFile,outputShapefile,expression)
        trimFields(outputShapefile)
        
    # for each traffic level cutoff, create a subset shapefile or roads that satisfy the range
    for cutoffIndex in range(len(taadtLevels)-1):
        outputShapefile = ROAD_ANNUAL_FOLDER + "/taadt" + str(cutoffIndex) + ".shp"
        masterList.append(outputShapefile)
        if not os.path.exists(outputShapefile):
            lowerCutoff = aadtLevels[cutoffIndex]
            upperCutoff = aadtLevels[cutoffIndex+1]
            var = "aadt_t" + str(year) 
            expression = var + " > " + str(lowerCutoff) 
            arcpy.Select_analysis(masterRoadFile,outputShapefile,expression)
        trimFields(outputShapefile)
    mergedShapefile = ROAD_ANNUAL_FOLDER + "/merged" + str(year) + ".shp"
    
    # merge all roads that satisfy at leat one cutoff range.  Used to restrict births near 500m of one of these roads
    if not os.path.exists(mergedShapefile):
        arcpy.Merge_management(masterList,mergedShapefile)

In [ ]:
for year in range(2007,2016):
    createAnnualFiles(year)

In [ ]:
YEARS = list(range(2007,2016))
ROAD_FOLDER = const.WIND_FOLDER + "roads/"
aadtLevels = [25000,50000,1000000000] # cutoffs for general traffic
taadtLevels = [5000,10000,1000000000] # cutoffs for truck traffic

# Calculate buffer average for each wind
This section should be done on the server due to the confidentiality.

In [ ]:
# Summary: given a shapefile containing wind, road, and shielding estimates,
#          calculate buffer averages for each exposure metric

In [ ]:
# Steps include:
# 1) exporting shapefile into csv format
# 2) renaming variables to match the final data dictionary
# 3) calculating buffer averages 
# 4) exporting buffer averages to csv

In [ ]:
# needed when using the ArcGIS license for a large # of parallel threads
while(sucessfulImport == False):
    try:
        #print("try again")
        import arcpy
        from arcpy.ia import * 
        from arcpy import env
        from arcpy.sa import *
        arcpy.env.overwriteOutput = True
        arcpy.CheckOutExtension("Spatial")
        sucessfulImport = True
    except Exception as e:
        print("couldn't import arcgis: %s" %(str(e)))
        time.sleep(1)


YEAR = 2010

In [ ]:
# given one shapefile containing exposure metrics for a single materal residence,
# transform results to a dataframe and then export to a csv
# also, reanme variables to match the final data dictionary